In [5]:
from qiskit import QuantumCircuit
from qiskit.providers.aer import Aer
from qiskit import execute
from qiskit.visualization import plot_histogram
from math import pi

ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [ ]:
def oracle(circuit, qubits):
    """Oracle marking the |11> state as the solution."""
    circuit.cz(qubits[0], qubits[1])

def diffuser(nqubits):
    """Grover's diffuser."""
    qc = QuantumCircuit(nqubits)
    qc.h(range(nqubits))
    qc.x(range(nqubits))
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1)
    qc.h(nqubits - 1)
    qc.x(range(nqubits))
    qc.h(range(nqubits))
    return qc

In [ ]:
# Number of qubits
nqubits = 2
grover = QuantumCircuit(nqubits)

# Initialize in the superposition state
grover.h(range(nqubits))

# Apply the oracle (mark |11> as the solution)
oracle(grover, [0, 1])

# Apply the Grover diffuser
grover.compose(diffuser(nqubits), inplace=True)

# Measure the result
grover.measure_all()

# Simulate the circuit
simulator = Aer.get_backend('aer_simulator')
job = execute(grover, simulator, shots=1024)
result = job.result()
counts = result.get_counts(grover)

# Display results
print("Measurement Results:")
print(counts)
grover.draw('mpl').show()
plot_histogram(counts).show()